## Problem 1: Visualise data in an interactive map (10 points)


Create an *interactive* **thematic map** using the skills you learnt during lesson 5. You can use *Folium*,<br>
or any other suitable Python package - feel free to experiment).

The map should contain more than one layer (from at least two different data sets). Pay attention to <br>
classification and visualisation (appropriate colour scheme choice, etc.). Write your code in this <br>
notebook, and store the resulting maps in `.html` format (plus possibly additional files, depending on<br>
your approach) in the `NOTEBOOK_DIRECTORY / "docs"` folder.

### Topic of the map

- Use any of the data sets we have used during the course (travel time data, population grid), or <br>
  any other data set of your interest (for example, [open data sets from Statistics Finland](https://www.stat.fi/org/avoindata/index_en.html), <br>
  or the open geo-data from [the Helsinki Region Infoshare](https://hri.fi/en)). If you download <br>
  data sets, add, commit, and push them to the `DATA_DIRECTORY`. Be sure to include an attribution <br>
  to each data set in your output map!
- Feel free to start from and adapt examples from the course material! You can take one step further <br>
  and add additional analysis to, e.g., the shopping centre dominance or the population within vincinity <br>
  of certain addresses)
- This should **not** be an interactive version of your submission for *problem 1*! Create something new!

### Criteria

- The map has more than one data layer (for instance, the results of an analysis, and the road network <br>
  to illustrate a complex spatial pattern). If (and only if) you think it adds additional value, feel <br>
  free to add a base map (but a base map does not count as an own layer)
- The map portrays the output of an analysis or a classification that adds value beyond just displaying <br>
  the raw data
- The map follows good cartographic practices (appropriate map extent and zoom level, colour scheme, <br>
  legend and data sources, etc.)
- The map demonstrates the skills learnt during lesson 5, interactive elements (popups, etc.) are a plus.

### Output

- Save the map(s) in `.png` format in the `NOTEBOOK_DIRECTORY / "docs"` folder, remember to add, commit, <br>
  and push them to the GitHub repository
- Remember to also add, commit, and push input data sets and code

---

In [1]:
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"
OUTPUT_DIRECTORY = NOTEBOOK_PATH / "docs"

In [2]:
import folium
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from branca.colormap import linear

/tmp/ipykernel_3835/3559278912.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
# Loading data
shopping_centres = gpd.read_file(DATA_DIRECTORY / "shopping_centres.gpkg").to_crs("EPSG:4326")
shopping_centres = shopping_centres.drop(columns=["addr"])
roads_WFS_URL = (
    "https://kartta.hel.fi/ws/geoserver/avoindata/wfs"
    "?service=wfs"
    "&version=2.0.0"
    "&request=GetFeature"
    "&srsName=EPSG:3879"
    "&typeName={layer:s}"
).format(layer="avoindata:Seutukartta_liikenne_paatiet")

population_grid = (
    gpd.read_file(
        "https://kartta.hsy.fi/geoserver/wfs"
        "?service=wfs"
        "&version=2.0.0"
        "&request=GetFeature"
        "&typeName=asuminen_ja_maankaytto:Vaestotietoruudukko_2020"
        "&srsName=EPSG:4326"
        "&bbox=24.6,60.1,25.2,60.4,EPSG:4326"
    )
    .set_crs("EPSG:4326")
)

population_grid = population_grid[["index", "asukkaita", "geometry"]]
population_grid = population_grid.rename(columns={
    "asukkaita": "population"
})
population_grid["id"] = population_grid.index.astype(str)

# roads df
roads_df = gpd.read_file(roads_WFS_URL).set_crs("EPSG:3879").to_crs("EPSG:4326")
roads_df = roads_df.dropna()

In [4]:
# Creating 2 km buffers
shopping_centres["two_km_buffer"] = shopping_centres["geometry"].to_crs("EPSG:3857").buffer(2000).to_crs("EPSG:4326")
shopping_centres = shopping_centres.set_geometry("two_km_buffer")

In [5]:
# Perform a spatial join to get gridsquares intersecting the defined buffers 
pop_within_2km_range = population_grid.sjoin(
    shopping_centres,
    how="left",
    predicate="intersects"
)

# Drop non intersecting grid squares and irrelevant columns
pop_within_2km_range = pop_within_2km_range.dropna()
pop_within_2km_range = pop_within_2km_range[["population", "name"]]

# Get the su of population within each 2km buffer around each shopping center
pop_within_2km_range = pop_within_2km_range.groupby("name").sum().reset_index()

# set geometry back to the locations of shopping centers (points)
shopping_centres = shopping_centres.set_geometry("geometry")

# get the aggregate population data inside the defined buffers
shopping_centres = shopping_centres.set_index("name").join(pop_within_2km_range.set_index("name")).reset_index()

In [6]:
# ADD YOUR OWN CODE HERE

interactive_map = folium.Map(
    width='50%',
    height='50%',
    location=(60.225, 24.920),
    zoom_start=11,
    control_scale=True
)

sh_centers_fg = folium.FeatureGroup("Shopping Centers")

for lat, lon, name, pop in zip(shopping_centres.geometry.y, shopping_centres.geometry.x, shopping_centres.name, shopping_centres.population):
    sh_centers_fg.add_child(
                                folium.Marker(
                                    location=[lat,lon],
                                    tooltip=f'{name}:<br> ~{pop} residents within 2 km',
                                    icon=folium.Icon(icon="shopping-cart")))

interactive_map.add_child(sh_centers_fg)

# Create a FeatureGroup to hold the linestring
roads_fg = folium.FeatureGroup(name='Roads')

for index, row in roads_df.iterrows():
    folium.GeoJson(row['geometry']).add_to(roads_fg)

# Add the FeatureGroup to the map
roads_fg.add_to(interactive_map)

choropleth_fg = folium.Choropleth(
    geo_data=shopping_centres.two_km_buffer,  # Replace with your GeoJSON data
    data=shopping_centres.population,
    key_on="feature.id",
    fill_color="YlGn",
    bins=4,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population within 2 km of shopping center',
).add_to(interactive_map)

# Displays a choropleth map indicating the population density inside each buffer of 2km around each shopping center 
interactive_map 

In [7]:
# Save
interactive_map.save(OUTPUT_DIRECTORY / "base-map.html")